In [1]:
import tensorflow as tf 
from tensorflow import keras 

import numpy as np 
import matplotlib.pyplot as plt

In [2]:
# 파일 불러오기
import os, re, glob
import cv2

image_size = 32

groups_folder_path = './Gramian/'

X_r = []
X_a = []
y = []

for label in ['Bi', 'Tri']:
    
    for feature in ['RMS', 'Angle']:
    
    
        for top, dir, f in os.walk(groups_folder_path+label+"/"+feature+"/"):
            for filename in f:
                img = cv2.imread(groups_folder_path+label+"/"+feature+"/"+filename)
                img = cv2.resize(img, None, fx=image_size/img.shape[0], fy=image_size/img.shape[1])
                
                if feature == 'RMS':
                    X_r.append(img/256)
                    
                elif feature == 'Angle':
                    X_a.append(img/256)   
                    
                    if label == 'Bi':
                        y.append(2)
                    else:
                        y.append(3)

In [3]:
print(X_r[0].shape)

(16, 64, 3)


In [4]:
X = []

for r, a in zip(X_r, X_a):
    img = []
    for i in range(len(r)):
        row = np.concatenate((r[i], a[i]), axis=1)
        img.append(row)
    X.append(img)
    
X = np.array(X)
y = np.array(y)

In [5]:
# train, test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
print(X_train.shape)

(16000, 16, 64, 6)


In [7]:
import collections

collections.Counter(y_train)

Counter({3: 8000, 2: 8000})

In [8]:
# validation split

X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.1, random_state=10, stratify=y_train)

In [9]:
print(X_train.shape, X_validation.shape)
print(collections.Counter(y_train))

(14400, 16, 64, 6) (1600, 16, 64, 6)
Counter({2: 7200, 3: 7200})


In [10]:
# plt.figure(figsize=(32,32))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(X_train[i], cmap=plt.cm.binary)
#     # The CIFAR labels happen to be arrays, 
#     # which is why you need the extra index
#     plt.xlabel(y_train[i])
# plt.show()

In [11]:
AlexNet = keras.models.Sequential([
        keras.layers.Conv2D(256, (6, 6), activation='relu', input_shape=X_train[0].shape), 
        keras.layers.MaxPooling2D( (2, 2) ), 
        keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'), 
        keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'), 
        keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'), 
        keras.layers.MaxPooling2D( (2, 2) ), 
        keras.layers.Flatten(), 
        keras.layers.Dense(4096, activation='relu'), 
        keras.layers.Dense(4096, activation='relu'), 
        keras.layers.Dense(len(y_train), activation='softmax') 
])

In [12]:
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 11, 59, 256)       55552     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 5, 29, 256)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 29, 384)        885120    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 29, 384)        1327488   
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 29, 256)        884992    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 14, 256)        0         
_________________________________________________________________
flatten (Flatten)            (None, 7168)              0

In [13]:
AlexNet.compile(loss=keras.losses.sparse_categorical_crossentropy, 
              optimizer=keras.optimizers.SGD(learning_rate=0.01), 
              metrics=[keras.metrics.sparse_categorical_accuracy])

In [14]:
AlexNet_history = AlexNet.fit(X_train, y_train, epochs=30, 
                    validation_data=(X_validation, y_validation))

Train on 14400 samples, validate on 1600 samples
Epoch 1/30
14400/14400 [==============================] - 325s 23ms/sample - loss: 1.2993 - sparse_categorical_accuracy: 0.5009 - val_loss: 0.7016 - val_sparse_categorical_accuracy: 0.5000
Epoch 2/30
14400/14400 [==============================] - 337s 23ms/sample - loss: 0.7178 - sparse_categorical_accuracy: 0.5066 - val_loss: 0.7126 - val_sparse_categorical_accuracy: 0.5000
Epoch 3/30
14400/14400 [==============================] - 337s 23ms/sample - loss: 0.7077 - sparse_categorical_accuracy: 0.5138 - val_loss: 0.7011 - val_sparse_categorical_accuracy: 0.5000
Epoch 4/30
14400/14400 [==============================] - 329s 23ms/sample - loss: 0.7069 - sparse_categorical_accuracy: 0.5155 - val_loss: 0.6929 - val_sparse_categorical_accuracy: 0.5494
Epoch 5/30
14400/14400 [==============================] - 335s 23ms/sample - loss: 0.7048 - sparse_categorical_accuracy: 0.5162 - val_loss: 0.7139 - val_sparse_categorical_accuracy: 0.5000
Epoch 

In [15]:
AlexNet_history = AlexNet.fit(X_train, y_train, epochs=5, 
                    validation_data=(X_test, y_test))

Train on 14400 samples, validate on 4000 samples
Epoch 1/5
14400/14400 [==============================] - 321s 22ms/sample - loss: 0.4382 - sparse_categorical_accuracy: 0.7926 - val_loss: 0.5076 - val_sparse_categorical_accuracy: 0.7667
Epoch 2/5
14400/14400 [==============================] - 324s 22ms/sample - loss: 0.4395 - sparse_categorical_accuracy: 0.7922 - val_loss: 0.4736 - val_sparse_categorical_accuracy: 0.7753
Epoch 3/5
14400/14400 [==============================] - 325s 23ms/sample - loss: 0.4267 - sparse_categorical_accuracy: 0.7999 - val_loss: 0.4578 - val_sparse_categorical_accuracy: 0.7878
Epoch 4/5
14400/14400 [==============================] - 321s 22ms/sample - loss: 0.4204 - sparse_categorical_accuracy: 0.8030 - val_loss: 0.4586 - val_sparse_categorical_accuracy: 0.7878
Epoch 5/5
14400/14400 [==============================] - 320s 22ms/sample - loss: 0.4104 - sparse_categorical_accuracy: 0.8122 - val_loss: 0.4962 - val_sparse_categorical_accuracy: 0.7678
